In [1]:
import os
os.chdir("..")
pwd = %pwd
pwd[12:]

'PJATK/semestr4/ADD/Covid19_Prediction'

In [2]:
import pandas as pd

In [3]:
import json
with open("data/covid_data.json") as f:
    d = json.load(f)

In [4]:
import numpy as np
import os
import typing

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
def preprocess_data(data: dict):
    df = pd.DataFrame(data["records"])
    df = df[df["countryterritoryCode"] == "POL"]
    df['dateRep'] = pd.to_datetime(df['dateRep'], format="%d/%m/%Y")
    df = df.sort_values("dateRep")
    cleaned_data = df.drop(["countryterritoryCode", "continentExp", "geoId", "countriesAndTerritories", "dateRep", "day", "month", "year", "popData2020", "deaths"], axis=1)
    cleaned_data['cases_next_day'] = df['cases'].shift(-1)
    x = cleaned_data.drop(["cases_next_day"],  axis=1).values[:-1]
    y = cleaned_data["cases_next_day"].values[:-1]
    return train_test_split(x, y, test_size=0.3, shuffle=False)

In [7]:
x_train, x_val, y_train, y_val = preprocess_data(d)

In [8]:
from sklearn.ensemble import RandomForestRegressor

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [88]:
model = RandomForestRegressor(n_jobs=-1, random_state=420)
estimators = 90
model.set_params(n_estimators=estimators)


pipeline = Pipeline([
    ('scaler2' , StandardScaler()),
    ('RandomForestRegressor: ', model)])
pipeline.fit(x_train , y_train)
prediction = pipeline.predict(x_val)

pipeline.fit(x_train, y_train)
pipeline.score(x_val, y_val)

0.8134111689164283

In [81]:
pp = pipeline.predict(x_val)

In [82]:
pipeline.predict([x_val[0]])

array([8692.16666667])

In [83]:
from sklearn.metrics import mean_squared_error, r2_score

In [84]:
mean_squared_error(y_val, pp, squared=False) # squared: If True returns MSE value, if False returns RMSE value. (we want RMSE)

5094.754797470026

In [85]:
r2_score(y_val, pp)

0.8134252950659202